# Descrição do programa

- Esse programa baixa dados eleitorais de 2020 e cria variável eleitorais a nível de candidatos

# TODO

- Ajeitar o 3 lugar. Quando só ha 2 candidatos, acho que a diferença do 3 lugar está ficando 0

# Configurando o ambiente

In [3]:
import pandas as pd
import numpy as np
import basedosdados as bd

work_dir   = "C:/Users/gabri/OneDrive/Gabriel/Insper/Tese/Engenheiros/replication_code/1_create_electoral_data/"
output_dir = "C:/Users/gabri/OneDrive/Gabriel/Insper/Tese/Engenheiros/replication_code/1_create_electoral_data/output/"

# Procurando candidatos eleitos na BD+

In [4]:
query = """ SELECT
                *         
            FROM `basedosdados.br_tse_eleicoes.candidatos` candidatos
            INNER JOIN `basedosdados.br_tse_eleicoes.resultados_candidato` resultados
            ON candidatos.id_candidato_bd = resultados.id_candidato_bd
            WHERE candidatos.cargo = 'prefeito'
            AND resultados.cargo = 'prefeito'
            AND resultados.tipo_eleicao = 'eleicao ordinaria'
            AND candidatos.tipo_eleicao = 'eleicao ordinaria'
            AND resultados.turno = 1
            AND (resultados.ano = 2020)
            AND (candidatos.ano = 2020) """

df_candidatos = bd.read_sql(query, billing_project_id= "econometria-314719")


Downloading: 100%|███████████████████████████████████████████████████████████| 17984/17984 [00:13<00:00, 1313.57rows/s]


### salvando

In [5]:
df_candidatos.to_csv(output_dir + 'data/df_candidatos_2020.csv', sep = ";", index = False, encoding = "utf-8-sig")

### abrindo

In [6]:
#df_candidatos = pd.read_csv(output_dir + "data/df_candidatos.csv", sep = ";", dtype = {"id_municipio": str, "id_municipio_tse": str, "id_candidato_bd": str, "cpf": str})

In [7]:
len(df_candidatos)

17984

In [8]:
len(df_candidatos["id_municipio"].unique())

5552

- Ou seja, dos 5568 municípios brasileiros, apenas 16 não tiveram eleições ordinárias

### Excluindo municípios com apenas 1 candidato

In [9]:
df_candidatos['n_candidatos'] = df_candidatos.groupby(['id_municipio'])['id_municipio'].transform('count')

In [10]:
df_candidatos = df_candidatos[df_candidatos['n_candidatos'] > 1] 

In [11]:
len(df_candidatos[df_candidatos["ano"] == 2020]["id_municipio"].unique())

5341

- Ou seja, em torno de 211 municípios só tinham 1 candidato

### Mantendo apenas municípios onde candidato foi eleito no primeiro turno

In [12]:
df_candidatos["decisao_primeiro_turno"] = df_candidatos["resultado"] == 'eleito'

In [13]:
df_candidatos["decisao_primeiro_turno"] = df_candidatos.groupby(['id_municipio']).decisao_primeiro_turno.transform('sum')

In [14]:
df_candidatos = df_candidatos[df_candidatos["decisao_primeiro_turno"] == 1]

In [15]:
df_candidatos.resultado.value_counts()

nao eleito    11824
eleito         5236
Name: resultado, dtype: int64

In [16]:
len(df_candidatos["id_municipio"].unique())

5236

- Ou seja, em torno de 105 municípios não tiveram decisão no 1 turno

### Criando variável de percentual de votos por candidato

In [17]:
df_candidatos["total_votos"] = df_candidatos.groupby(['id_municipio']).votos.transform('sum')

In [18]:
df_candidatos["%votos"] = (df_candidatos.votos / df_candidatos.total_votos)

### Criando variável para segundo lugar

In [19]:
df_candidatos["2_lugar"] = df_candidatos[df_candidatos["resultado"] == 'nao eleito'].groupby(['id_municipio']).votos.transform('max')

In [20]:
df_candidatos["2_lugar"] = df_candidatos.votos == df_candidatos["2_lugar"]

### Criando variável para terceiro lugar

In [21]:
df_candidatos["3_lugar"] = df_candidatos[(df_candidatos["resultado"] == 'nao eleito') & (df_candidatos["2_lugar"] == False)].groupby(['id_municipio', 'ano']).votos.transform('max')

In [22]:
df_candidatos["3_lugar"] = df_candidatos.votos == df_candidatos["3_lugar"]

### Mantendo apenas o eleito, o segundo e o terceiro lugar

In [23]:
df_candidatos = df_candidatos[(df_candidatos["resultado"] == 'eleito') | (df_candidatos["2_lugar"] == True) | (df_candidatos["3_lugar"] == True)]

### Criando variável de diferença entre primeiro e segundo candidatos e entre primeiro e terceiro candidatos

In [24]:
dif_segundo = pd.DataFrame(df_candidatos.groupby(['id_municipio'])['%votos'].apply(lambda x: x.nlargest(2).iloc[0] - x.nlargest(2).iloc[-1])).reset_index()

In [25]:
dif_segundo = dif_segundo.rename(columns = {"%votos": "dif_votos_2_lugar"})

In [26]:
df_candidatos = pd.merge(df_candidatos, dif_segundo, how = "outer", on = "id_municipio", indicator = True)

In [27]:
df_candidatos._merge.value_counts()

both          13585
left_only         0
right_only        0
Name: _merge, dtype: int64

In [28]:
df_candidatos = df_candidatos.drop(columns = "_merge")

In [29]:
dif_terceiros = pd.DataFrame(df_candidatos.groupby("id_municipio")['%votos'].apply(lambda x: x.nlargest(3).iloc[0] - x.nlargest(3).iloc[-1])).reset_index()

In [30]:
dif_terceiros = dif_terceiros.rename(columns = {"%votos": "dif_votos_3_lugar"})

In [31]:
df_candidatos = pd.merge(df_candidatos, dif_terceiros, how = "outer", on = "id_municipio", indicator = True)

In [32]:
df_candidatos._merge.value_counts()

both          13585
left_only         0
right_only        0
Name: _merge, dtype: int64

In [33]:
df_candidatos.describe()

,ano,idade,ano_1,turno,votos,n_candidatos,decisao_primeiro_turno,total_votos,%votos,dif_votos_2_lugar,dif_votos_3_lugar
count,13585.0,13585.000000,13585.0,13585.0,13585.000000,13585.000000,13585.0,1.358500e+04,13585.000000,13585.000000,13585.000000
mean,2020.0,49.763563,2020.0,1.0,5190.512256,3.454987,1.0,1.559556e+04,0.376330,0.178352,0.332882
std,0.0,11.006271,0.0,0.0,14869.736310,1.639219,0.0,3.855184e+04,0.192395,0.171953,0.201757
min,2020.0,21.000000,2020.0,1.0,2.000000,2.000000,1.0,6.160000e+02,0.000252,0.000000,0.000000
25%,2020.0,42.000000,2020.0,1.0,1260.000000,2.000000,1.0,4.075000e+03,0.238779,0.056080,0.156284
50%,2020.0,50.000000,2020.0,1.0,2526.000000,3.000000,1.0,7.476000e+03,0.406652,0.125111,0.337898
75%,2020.0,57.000000,2020.0,1.0,5300.000000,4.000000,1.0,1.494500e+04,0.509783,0.243022,0.490029
max,2020.0,95.000000,2020.0,1.0,784307.000000,16.000000,1.0,1.237764e+06,0.998069,0.996139,0.996139


In [34]:
df_candidatos.loc[df_candidatos["n_candidatos"] == 2, "dif_votos_3_lugar"] = None # Colocando Null na diferença entre 1 e 3 candidato nos municípios que só têm 2 candidatos

In [35]:
df_candidatos.describe()

,ano,idade,ano_1,turno,votos,n_candidatos,decisao_primeiro_turno,total_votos,%votos,dif_votos_2_lugar,dif_votos_3_lugar
count,13585.0,13585.000000,13585.0,13585.0,13585.000000,13585.000000,13585.0,1.358500e+04,13585.000000,13585.000000,9337.000000
mean,2020.0,49.763563,2020.0,1.0,5190.512256,3.454987,1.0,1.559556e+04,0.376330,0.178352,0.393450
std,0.0,11.006271,0.0,0.0,14869.736310,1.639219,0.0,3.855184e+04,0.192395,0.171953,0.174863
min,2020.0,21.000000,2020.0,1.0,2.000000,2.000000,1.0,6.160000e+02,0.000252,0.000000,0.007768
25%,2020.0,42.000000,2020.0,1.0,1260.000000,2.000000,1.0,4.075000e+03,0.238779,0.056080,0.259222
50%,2020.0,50.000000,2020.0,1.0,2526.000000,3.000000,1.0,7.476000e+03,0.406652,0.125111,0.416296
75%,2020.0,57.000000,2020.0,1.0,5300.000000,4.000000,1.0,1.494500e+04,0.509783,0.243022,0.516148
max,2020.0,95.000000,2020.0,1.0,784307.000000,16.000000,1.0,1.237764e+06,0.998069,0.996139,0.986481


# Salvando

In [36]:
df_candidatos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13585 entries, 0 to 13584
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ano                     13585 non-null  int64         
 1   tipo_eleicao            13585 non-null  object        
 2   sigla_uf                13585 non-null  object        
 3   id_municipio            13585 non-null  object        
 4   id_municipio_tse        13585 non-null  object        
 5   id_candidato_bd         13585 non-null  object        
 6   cpf                     13585 non-null  object        
 7   titulo_eleitoral        13585 non-null  object        
 8   sequencial              13585 non-null  object        
 9   numero                  13585 non-null  object        
 10  nome                    13585 non-null  object        
 11  nome_urna               13585 non-null  object        
 12  numero_partido          13585 non-null  object

In [37]:
df_candidatos = df_candidatos.loc[:,["ano","tipo_eleicao", "sigla_uf","id_municipio","cpf","nome","nome_urna","numero_partido","sigla_partido","cargo","situacao","ocupacao","idade","genero","instrucao","raca","resultado","2_lugar","3_lugar","dif_votos_2_lugar","dif_votos_3_lugar"]]

In [38]:
df_candidatos.to_csv(output_dir + "data/df_candidatos_2020_clean.csv", sep = ";", index = False, encoding = "utf-8-sig")

In [39]:
cpfs_rdd = tuple(df_candidatos.cpf.unique())

In [40]:
len(cpfs_rdd)

13585